# Pitch angle scattering
Look at how electron scatters change the pitch angles of the electrons.

In [ ]:
# Basic imports and settings
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.constants as const
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
# Import lmfit
import lmfit

import time

In [ ]:
# Import the cross-section classes
from src.Rudd1991 import RuddXSec
from src.Mott import MottXSec
import src.Constants as myconst

In [ ]:
# Make matplotlib figures better and allow latex
plt.rcParams['text.usetex'] = True
# Make figures bigger
plt.rcParams['figure.figsize'] = (8, 5)
# Make font sizes bigger
plt.rcParams.update({'font.size': 14})

## Trapping time of endpoint electrons
Look at generating monoenergetic endpoint electrons with $T = 18575$ keV. Assume these electrons are always in some flat region of the trap and see how many scatters they undergo before being ejected from the trap.

### Verify the initial electron pitch angle distribution

In [ ]:
NELECTRONS = 2000
# Generate isotropic starting directions
theta = np.arccos(2 * np.random.rand(NELECTRONS) - 1)
phi = 2 * np.pi * np.random.rand(NELECTRONS)
xVelGen = np.sin(theta) * np.cos(phi)
yVelGen = np.sin(theta) * np.sin(phi)
zVelGen = np.cos(theta)
radialVelGen = np.sqrt(xVelGen**2 + yVelGen**2)
pitchAngleGen = np.arctan(radialVelGen / zVelGen)

# Plot these points in 3D
fig = plt.figure(figsize=(18, 5))
ax = fig.add_subplot(121, projection='3d')
ax.scatter(xVelGen, yVelGen, zVelGen)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
ax.set_title('Initial electron velocities vectors')

ax2 = fig.add_subplot(122)
# Plot the pitch angle distribution histogram with error bars
y, bin_edges = np.histogram(pitchAngleGen * 180 / np.pi, bins='auto')
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
ax2.bar(bin_centers, y, width=bin_edges[1] - bin_edges[0], yerr=np.sqrt(y))
plt.xlabel('Pitch angle [degrees]')
plt.ylabel('Number of electrons')
plt.title('Initial pitch angle distribution')
plt.xlim([-90, 90])
plt.show()

### Estimating electron trapping times (single energy)
Take an electron and see how long it propagates for before leaving the trap. 

In [ ]:
def CalcCDF_Rudd_SDCS_W(T):
    """
    Calculate the cumulative distribution function for the singly-differential 
    ionisation cross-section in W. 

    Parameters:
    -----------
    T : (float) The kinetic energy of the electron in eV.
    """
    # First, the values of W to be probed
    diffArr = np.logspace(-3, np.log10(T / 2.0), 100)
    WArr = (T - myconst.IONIZATIONENERGYH) * np.ones_like(diffArr) - diffArr

    # Calculate the cross-sections
    sdcsW = RuddXSec(T).SinglyDifferentialXSec_W(WArr)

    # Add a zero point to the end of the arrays
    WArr = np.append(WArr, 0.0)
    sdcsW = np.append(sdcsW, 0.0)
    # Reverse arrays
    sdcsW = np.flip(sdcsW)
    WArr = np.flip(WArr)

    # Get the normalisation constant
    sdcsWIntegral = np.trapz(sdcsW, x=WArr)

    # Create an array to store the CDF
    cdf = np.zeros_like(WArr)
    for i, W in enumerate(WArr):
        cdf[i] = np.trapz(sdcsW[:i], x=WArr[:i])

    cdf /= cdf[-1]

    return WArr, cdf

def CalcCDF_DDCS_Theta(T, W):
    """
    Calculate the CDF for the doubly-differential cross-section in theta for a
    given value of W and T.

    Parameters
    ----------
    T: Incident energy in eV
    W: Energy transfer in eV
    """
    
    nPnts = 100
    theta = np.logspace(-5, np.log10(np.pi / 2.0), nPnts)
    ddcsTheta = np.zeros_like(theta)
    for i in range(1, nPnts):
        ddcsTheta[i] = RuddXSec(T).DoublyDifferentialXSec(W, theta[i])

    cdf = np.zeros_like(theta)
    for i, th in enumerate(theta):
        cdf[i] = np.trapz(ddcsTheta[:i], x=theta[:i])

    cdf /= cdf[-1]

    return theta, cdf

def RotateToCoords(v, xAx, yAx, zAx):
    """
    Rotate a vector to a new coordinate system

    Parameters
    ----------
    v: Unit vector to be rotated
    """
    x1Prime = (xAx * v[0])[0] + (yAx * v[1])[0] + (zAx * v[2])[0]
    x2Prime = (xAx * v[0])[1] + (yAx * v[1])[1] + (zAx * v[2])[1]
    x3Prime = (xAx * v[0])[2] + (yAx * v[1])[2] + (zAx * v[2])[2]
    return np.array([x1Prime, x2Prime, x3Prime])

def GetScatteredVector(vIn, theta):
    """
    Calculation of the scattered velocity vector

    Parameters
    ----------
    vIn: Incident velocity vector
    theta: Scattering angle in radians
    """

    phiGen = 2.0 * np.pi * np.random.rand()

    # Original direction in global coordinates
    originalDir = vIn / np.linalg.norm(vIn)
    # Original direction in scattering frame
    oldDir = np.array([0.0, 0.0, 1.0])
    # New direction in scattering frame
    newDir = np.array([np.sin(theta) * np.cos(phiGen), 
                       np.sin(theta) * np.sin(phiGen), 
                       np.cos(theta)])

    # Define other axes
    ax2 = np.array([-originalDir[1] / originalDir[0], 1.0, 0.0])  
    ax2 /= np.linalg.norm(ax2)
    ax3 = np.cross(ax2, originalDir)
    ax3 /= np.linalg.norm(ax3)  

    return RotateToCoords(newDir, ax2, ax3, originalDir)

In [ ]:
# Now create code for a CDF of the Mott scattering cross-section
def CalcCDF_Mott_Theta(T):
    """
    Calculate the CDF for the Mott scattering cross-section in theta for a given
    value of T.

    Parameters
    ----------
    T: Incident energy in eV
    """
    nPnts = 300
    thetaArr = np.logspace(-5, np.log10(np.pi * 0.99999), nPnts)
    mott = MottXSec(T)
    sdcsTheta = mott.SinglyDifferentialXSec_theta(thetaArr)

    cdf = np.zeros_like(thetaArr)
    for i, th in enumerate(thetaArr):
        cdf[i] = np.trapz(sdcsTheta[:i], x=thetaArr[:i])

    cdf /= cdf[-1]

    return thetaArr, cdf

Draw some plots of the differential cross-sections and the associated cumulative distribution functions.

In [ ]:
# Some diagnostic plots for an endpoint electron
EKE = 18.575e3  # eV
rudd = RuddXSec(EKE)

# Array of differences to use
diffArr = np.logspace(-3, np.log10(EKE / 2.0), 100)
wArr = (EKE - myconst.IONIZATIONENERGYH) * np.ones_like(diffArr) - diffArr
# Singly differential (in W) cross-section
sdcsW = rudd.SinglyDifferentialXSec_W(wArr)

plt.figure(figsize=(19, 10))
plt.subplot(221)
plt.plot(wArr / 1e3, sdcsW)
plt.xlabel('$W$ [keV]')
plt.ylabel('$d\sigma/dW$ [m$^2$ eV$^{-1}$]')
plt.yscale('log')
plt.xlim([(EKE - myconst.IONIZATIONENERGYH) / 2e3, EKE / 1e3])
plt.title(f"T = {EKE / 1e3:.3f} keV")

# Now calculate and plot the CDF for this distribution
wArr, cdfW = CalcCDF_Rudd_SDCS_W(EKE)
plt.twinx()
plt.plot(wArr / 1e3, cdfW, color='r', linestyle='--', label='CDF')
plt.ylabel('F($W$)')
plt.yscale('log')
plt.ylim([cdfW[2], 1.0])
plt.legend()

plt.subplot(222)
secondaryEnergy = 3.0 # eV
chosenW = (EKE - myconst.IONIZATIONENERGYH) - secondaryEnergy
# Calculate the CDF for the DDCS in theta
thetaArr, cdfTheta = CalcCDF_DDCS_Theta(EKE, chosenW)
# Calculate the DDCS
ddcsTheta = rudd.DoublyDifferentialXSec(chosenW, thetaArr)

plt.plot(thetaArr * 180 / np.pi, ddcsTheta)
plt.xlabel(r'$\theta$ [degrees]')
plt.xscale('log')
plt.xlim([6e-4, 90])
plt.ylabel(r'$d^2\sigma/dWd\theta$ [m$^2$ eV$^{-1}$ rad$^{-1}$]')
plt.title(f"T = {EKE / 1e3:.3f} keV, W = {chosenW / 1e3:.3f} keV")
plt.twinx()
plt.plot(thetaArr * 180 / np.pi, cdfTheta, color='r', linestyle='--', 
         label='CDF')
plt.ylabel(r'F($\theta$)')
plt.legend()

plt.subplot(223)
thetaArr, cdfMottTheta = CalcCDF_Mott_Theta(EKE)
mott = MottXSec(EKE)
sdcsTheta = mott.SinglyDifferentialXSec_theta(thetaArr)
plt.plot(thetaArr * 180 / np.pi, sdcsTheta)
plt.xlabel(r'$\theta$')
plt.ylabel(r'$d\sigma/d\cos \theta$ [m$^2$]')
plt.xlim([1e-4, 180.0])
plt.xscale('log')
plt.yscale('log')
plt.twinx()
plt.plot(thetaArr * 180 / np.pi, cdfMottTheta, color='r', linestyle='--', 
         label='CDF')
plt.ylabel(r'F($\theta$)')
plt.legend()
plt.yscale('log')

plt.show()

Set up a function which, for a given electron energy, calculates the trapping time, the number of total scatters and the number of secondary electrons produced.

In [ ]:
TATOMDENS = 1e18 # atoms per m^3
def PropagateElectron(T: float, initialPitchAngle: float, minPitchAngle: float):
    """
    Propagate an electron with a given energy and pitch angle until it escapes 
    the trap.

    Parameters
    ----------
    T: (float) The initial kinetic energy of the electron in eV
    initialPitchAngle: (float) The initial pitch angle of the electron in radians
    minPitchAngle: (float) The minimum pitch angle which is trapped in radians

    Returns
    -------
    trappedTime: (float) The time the electron was trapped in seconds
    nScatters: (int) The number of scattering events
    nSecondaries: (int) The number of secondary electrons produced
    """
    
    trappedTime = 0.0
    nScatters = 0
    nSecondaries = 0

    pitchAngle = initialPitchAngle
    ke = T

    while abs(pitchAngle) > minPitchAngle:
        gamma = 1.0 + ke * const.e / (const.m_e * const.c**2)
        beta = np.sqrt(1.0 - 1.0 / gamma**2)
        speed = beta * const.c
        # Generate a random cyclotron phase
        cyclotronPhase = 2.0 * np.pi * np.random.rand()
        # Calculate the velocity vector
        velocity = speed * np.array([np.sin(pitchAngle) * np.cos(cyclotronPhase),
                                    np.sin(pitchAngle) * np.sin(cyclotronPhase),
                                    np.cos(pitchAngle)])
        # Get cross-sections
        rudd = RuddXSec(ke)
        mott = MottXSec(ke)
        totalXSec = rudd.TotalXSec() + mott.TotalXSec()
        # Get the mean free path
        meanFreePath = 1.0 / (TATOMDENS * totalXSec)
        # Get the time to the next ionisation event
        trappedTime += np.random.exponential(meanFreePath / speed)

        # Determine if this is a elastic or ionisation event
        if np.random.rand() < mott.TotalXSec() / totalXSec:
            # Elastic scattering event
            nScatters += 1
            # Get the new pitch angle
            thetaArr, cdfMottTheta = CalcCDF_Mott_Theta(ke)
            scatteringAngle = np.interp(np.random.rand(), cdfMottTheta, thetaArr)
            # Get the new velocity vector direction
            vDir = GetScatteredVector(velocity, scatteringAngle)
            # Update the pitch angle
            pitchAngle = np.arctan(np.sqrt(vDir[0]**2 + vDir[1]**2) / vDir[2])

        else:
            # Ionisation event
            nScatters += 1
            nSecondaries += 1
            # Get the energy of the secondary electron
            wArr, cdfRuddW = CalcCDF_Rudd_SDCS_W(ke)
            sampledW = np.interp(np.random.rand(), cdfRuddW, wArr)
            # Now get the scattering angle
            thetaRuddArr, cdfRuddTheta = CalcCDF_DDCS_Theta(ke, sampledW)
            scatteringAngle = np.interp(np.random.rand(), cdfRuddTheta, 
                                        thetaRuddArr)
            # Get the velocity vector of the secondary electron
            vDir = GetScatteredVector(velocity, scatteringAngle)
            pitchAngle = np.arctan(np.sqrt(vDir[0]**2 + vDir[1]**2) / vDir[2])
            ke = sampledW 

    return trappedTime, nScatters, nSecondaries

In [ ]:
# Test this out for an electron
initialPitchAngle = 86.0 * np.pi / 180.0
minPitchAngle = 85.0 * np.pi / 180.0
initialKE = 18.575e3
trappedTime, nScatters, nSecondaries = PropagateElectron(initialKE, 
                                                         initialPitchAngle, 
                                                         minPitchAngle)
print(f'Electron trapped for {trappedTime * 1e6:.3f} us with {nScatters} scatters and produced {nSecondaries} secondaries')

For monoenergetic endpoint electrons how many scattters do we expect and secondaries do we expect to be produced?

In [ ]:
NELECTRONS = 1000
nGenerated = 0
minPitchAngle = 85.0 * np.pi / 180.0
initialKE = 18.575e3

trappedTimeArr = np.zeros(NELECTRONS)
nScattersArr = np.zeros(NELECTRONS)
nSecondariesArr = np.zeros(NELECTRONS)
initialAngleArr = np.zeros(NELECTRONS)
while nGenerated < NELECTRONS:
    # Generate a random direction
    theta = np.arccos(2 * np.random.rand() - 1)
    phi = 2 * np.pi * np.random.rand()
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    initialPitchAngle = np.arctan(np.sqrt(x**2 + y**2) / z)
    if abs(initialPitchAngle) >= minPitchAngle:
        nGenerated += 1
        trappedTime, nScatters, nSecondaries = PropagateElectron(
            initialKE, initialPitchAngle, minPitchAngle)
        initialAngleArr[nGenerated - 1] = initialPitchAngle
        trappedTimeArr[nGenerated - 1] = trappedTime
        nScattersArr[nGenerated - 1] = nScatters
        nSecondariesArr[nGenerated - 1] = nSecondaries

plt.figure(figsize=(18, 5))
plt.subplot(131)
plt.hist(trappedTimeArr * 1e6, bins='auto')
plt.xlabel(r'Trapped time [$\mu$s]')
plt.ylabel('Number of electrons')
plt.title(f'T = {initialKE / 1e3:.3f} keV')

plt.subplot(132)
plt.hist(nScattersArr, bins='auto')
plt.xlabel('Number of scatters')
plt.ylabel('Number of electrons')
plt.title(f'T = {initialKE / 1e3:.3f} keV')
plt.xlim([0, 20])

plt.subplot(133)
plt.hist(nSecondariesArr, bins='auto')
plt.xlabel('Number of secondaries')
plt.ylabel('Number of electrons')
plt.title(f'T = {initialKE / 1e3:.3f} keV')
plt.xlim([0, 20])

plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.hist(abs(initialAngleArr) * 180 / np.pi, bins='auto')
plt.xlabel('Initial pitch angle [degrees]')
plt.ylabel('Number of electrons')
plt.title(f'T = {initialKE / 1e3:.3f} keV')

plt.subplot(122)
plt.scatter(abs(initialAngleArr) * 180 / np.pi, nScattersArr)
plt.xlabel('Initial pitch angle [degrees]')
plt.ylabel('Number of scatters')
plt.title(f'T = {initialKE / 1e3:.3f} keV')

plt.show()

In [ ]:
# Now fit an exponential to the trapped time distribution to get the mean 
# trapped time
def exponential(x, a, tau):
    return a * np.exp(-x / tau)

# Get the bin centers and counts
tTimeCounts, tTimeEdges = np.histogram(trappedTimeArr * 1e6, bins='auto')
tTimeCenters = 0.5 * (tTimeEdges[1:] + tTimeEdges[:-1])
tTimeErrs = np.sqrt(tTimeCounts)
# Fit the exponential using lmfit
params = lmfit.Parameters()
params.add('a', value=np.max(tTimeCounts))
params.add('tau', value=np.mean(trappedTimeArr * 1e6))
tTimeModel = lmfit.Model(exponential)
result = tTimeModel.fit(tTimeCounts, x=tTimeCenters, params=params, 
                        weights=1 / tTimeErrs, nan_policy='omit')
# Get reduced chi-squared
reducedChi2 = result.redchi
# Get the fit values
aFit = result.params['a'].value
tauFit = result.params['tau'].value
# Get the fit errors
aErr = result.params['a'].stderr
tauErr = result.params['tau'].stderr

plt.figure()
plt.errorbar(tTimeCenters, tTimeCounts, yerr=tTimeErrs, fmt='o', label='Data')
plt.plot(tTimeCenters, result.best_fit, label=fr'Fit, $\tau = {tauFit:.0f} \pm {tauErr:.0f}$ $\mu$s, $\chi^2_\nu = ${reducedChi2:.2f}')
plt.xlabel(r'Trapped time [$\mu$s]')
plt.ylabel('Number of electrons')
plt.legend()
plt.show()

In [ ]:
# Now let's calculate this mean trapping time for a range of initial energies
initialKEArr = np.linspace(1e3, 19e3, 20)
lifetimeArr = np.zeros_like(initialKEArr)
lifetimeErrArr = np.zeros_like(initialKEArr)

for iKE, testKE in enumerate(initialKEArr):
    print(f'Testing {testKE / 1e3:.3f} keV')
    NELECTRONS = 300
    nGenerated = 0
    minPitchAngle = 85.0 * np.pi / 180.0

    tTimeTmpArr = np.zeros(NELECTRONS)
    nScattersTmpArr = np.zeros(NELECTRONS)
    nSecondariesTmpArr = np.zeros(NELECTRONS)
    
    while nGenerated < NELECTRONS:
        # Generate a random direction
        
        theta = np.arccos(2 * np.random.rand() - 1)
        phi = 2 * np.pi * np.random.rand()
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)
        initialPitchAngle = np.arctan(np.sqrt(x**2 + y**2) / z)
        if abs(initialPitchAngle) >= minPitchAngle:
            nGenerated += 1
            trappedTime, nScatters, nSecondaries = PropagateElectron(
                testKE, initialPitchAngle, minPitchAngle)
            tTimeTmpArr[nGenerated - 1] = trappedTime
            nScattersTmpArr[nGenerated - 1] = nScatters
            nSecondariesTmpArr[nGenerated - 1] = nSecondaries

    # Now fit an exponential to the trapped time distribution to get the mean
    # trapped time
    tTimeCounts, tTimeEdges = np.histogram(tTimeTmpArr * 1e6, bins='auto')
    tTimeCenters = 0.5 * (tTimeEdges[1:] + tTimeEdges[:-1])
    tTimeErrs = np.sqrt(tTimeCounts)
    # Fit the exponential using lmfit
    params = lmfit.Parameters()
    params.add('a', value=np.max(tTimeCounts))
    params.add('tau', value=np.mean(tTimeTmpArr * 1e6))
    tTimeModel = lmfit.Model(exponential)
    result = tTimeModel.fit(tTimeCounts, x=tTimeCenters, params=params, 
                            weights=1 / tTimeErrs, nan_policy='omit')
    # Get the fit values
    lifetimeArr[iKE] = result.params['tau'].value
    # Get the fit errors
    lifetimeErrArr[iKE] = result.params['tau'].stderr


In [ ]:
# Plot the mean trapping time as a function of initial energy
plt.figure()
plt.errorbar(initialKEArr / 1e3, lifetimeArr, yerr=lifetimeErrArr, fmt='o', 
             markersize=3, label='Data')
plt.xlabel('Initial energy [keV]')
plt.ylabel(r'Mean trapping time [$\mu$s]')

# Fit a straight line to the data
def linear(x, a, b):
    return a * x + b

params = lmfit.Parameters()
params.add('a', value=1.0)
params.add('b', value=0.0)
lifetimeModel = lmfit.Model(linear)
result = lifetimeModel.fit(lifetimeArr, x=initialKEArr, params=params, 
                           weights=1 / lifetimeErrArr, nan_policy='omit')
plt.plot(initialKEArr / 1e3, result.best_fit, 
         label=fr'Fit, $\chi^2_\nu = ${result.redchi:.2f}')
plt.legend()
# Write the fit parameters on the graph
aFit = result.params['a'].value
aFitErr = result.params['a'].stderr
bFit = result.params['b'].value
bFitErr = result.params['b'].stderr
plt.text(11.5, 400, r'$y = ax + b$')
plt.text(11.5, 325, fr'$a = ({aFit*1e3:.0f} \pm {aFitErr*1e3:.0f})$ $\mu$s/keV')
plt.text(11.5, 250, fr'$b = ({bFit:.0f} \pm {bFitErr:.0f})$ $\mu$s')
#plt.text(0.5, 0.9, fr'$\tau = {aFit:.2f} \pm {aFitErr:.2f}$ $\mu$s/keV', 
#         transform=plt.gca().transAxes)

# Plot the residuals underneath the main plot
plt.figure()
plt.errorbar(initialKEArr / 1e3, lifetimeArr - result.best_fit, 
             yerr=lifetimeErrArr, fmt='o')
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Initial energy [keV]')
plt.ylabel(r'Residuals [$\mu$s]')

plt.show()

## Comparison with electron production rates
We now want to investigate how these lifetimes compare with the expected electron production rates.

In [ ]:
eMass_eV = const.physical_constants['electron mass energy equivalent in MeV'][0] * 1e6 
G_F = const.physical_constants['Fermi coupling constant'][0] * 1e-18 # eV^-2
endpointEnergy = 18.575e3 # eV

def FermiFunction(beta):
  eta = const.alpha * 2.0 / beta
  return 2.0 * np.pi * eta / (1.0 - np.exp(-2.0 * np.pi * eta))

def TritiumDecayRate(T, endpointE=18.575e3):
  '''
  Calculate the differential decay rate of tritium in units of s^-1 eV^-1
  T: Electron kinetic energy in eV
  endpointE: The endpoint energy of the beta decay in eV
  '''
  # Calculate the beta factor
  beta = np.sqrt(1.0 - (eMass_eV / (T + eMass_eV))**2)
  p = np.sqrt(T**2 + 2.0 * eMass_eV * T)
  nuE = endpointE - T
  rate = G_F**2 * (0.97425)**2 * FermiFunction(beta) * (1.0 + 3.0 * (-1.2646)**2) * p * (T + eMass_eV) / (2.0 * np.pi**3) 
  
  return rate * nuE**2 / 6.58e-16

In [ ]:
# Plot the decay rate as a function of electron energy
TArr = np.linspace(1.0, endpointEnergy, 400)
plt.figure()
plt.plot(TArr / 1e3, TritiumDecayRate(TArr))
plt.xlabel('Electron kinetic energy [keV]')
plt.ylabel(r'$d\Gamma/dE$ [s$^{-1}$ eV$^{-1}$]')
plt.title(r'Tritium decay rate, $m_\beta=0$')
plt.xlim([0, endpointEnergy / 1e3])
plt.ylim([0, 2e-13])
plt.show()

# Tritium decay rate binned in energy
Use the same binning as used for the calculation of electron trapping times.

In [ ]:
keCentres = np.linspace(1e3, 19e3, 20)
keBinWidth = keCentres[1] - keCentres[0]
keEdges = np.append(initialKEArr - keBinWidth / 2, keCentres[-1] + keBinWidth / 2)
integratedDecayRates = np.zeros_like(keCentres)
for i, ke in enumerate(keCentres):
    integratedDecayRates[i] = quad(TritiumDecayRate, keEdges[i], keEdges[i+1])[0]

plt.figure()
plt.stairs(integratedDecayRates, keEdges / 1e3)
plt.xlabel('Initial electron energy [keV]')
plt.ylabel('Decay rate per atom [s$^{-1}$]')
plt.xlim([0, 19.5])
plt.ylim([0, 2e-10])
plt.show()

Now calculate for a volume of $10^{-3}~\text{m}^3$ and a number density of $10^{12}~\text{cm}^{-3}$ the number of electrons produced in each bin.

In [ ]:
VOLUME = 1e-3 # m^3
electronProdRates = integratedDecayRates * VOLUME * TATOMDENS
plt.figure(figsize=(14, 5))
plt.subplot(121)
plt.stairs(electronProdRates, keEdges / 1e3)
plt.xlabel('Initial electron energy [keV]')
plt.ylabel(fr'e$^-$ production rate [s$^{-1}$ / {keBinWidth:.0f} eV]')
plt.xlim([0, 19.5])
plt.title(r'Electron production rates, $V = 10^{-3}~\rm{m}^{3}$, $n = 10^{18}~\rm{m}^{-3}$')

# Calculate the trapped fraction in an 85 degree trap
plt.subplot(122)
dThetaMax = 5.0 * np.pi / 180.0
etaTrapped = np.sin(dThetaMax)
plt.stairs(electronProdRates * etaTrapped, keEdges / 1e3, color='black',
           label=f'Trapped e$^-$ production rate / {keBinWidth:.0f} eV')
plt.xlabel('Initial electron energy [keV]')
plt.ylabel(fr'Rate [s$^{-1}$]')
plt.xlim([0, 19.5])
plt.title(r'Electron production rates, $V = 10^{-3}~\rm{m}^{3}$, $n = 10^{18}~\rm{m}^{-3}$')
plt.errorbar(initialKEArr / 1e3, 1 / (lifetimeArr * 1e-6), color='r',
             yerr=(lifetimeErrArr*1e-6) / (lifetimeArr*1e-6)**2, fmt='o', 
             markersize=3, label=r'Trapped e$^-$ loss rate')
plt.legend()
plt.show()

# Simple simulation of the number of trapped electrons
With the parameters we've extracted, do some simple estimations of the number of trapped electrons after a given time.

In [ ]:
volume = 1e-3 # metres^3
atomDensity = 1e18 # atoms per metre^3 
nTAtoms = volume * atomDensity
oneYearInSeconds = 365.25 * 24 * 60 * 60 # seconds
halfLifeT = 12.32 * oneYearInSeconds # seconds
lifetimeT = halfLifeT / np.log(2) # seconds per atom
decayRateNAtoms = nTAtoms / lifetimeT # decays per second
print(f'Decay rate of {decayRateNAtoms:.2e} electrons/s')

# Generate a series of electron birth times along with their energies
timeWindow = 1.0  # seconds
meanEvents = decayRateNAtoms 
nDecays = np.random.poisson(meanEvents)
print(f'We have {nDecays} decays in our time window')

energies = np.linspace(1e-3, endpointEnergy, 1000)
decayRateArr = TritiumDecayRate(energies, endpointEnergy)
cdfDecayEnergy = np.zeros_like(energies)
for i, th in enumerate(cdfDecayEnergy):
    cdfDecayEnergy[i] = np.trapz(decayRateArr[:i], x=energies[:i])

cdfDecayEnergy /= cdfDecayEnergy[-1]

plt.figure()
plt.plot(energies / 1e3, cdfDecayEnergy)
plt.xlabel('Kinetic energy [eV]')
plt.ylabel('CDF')
plt.ylim(0, 1)
plt.xlim(0, endpointEnergy / 1e3)
plt.show()

trappingEvents = []

# Define a class to contain trapping events
class TrappingEvent:

    def __init__(self, energy, startTime, trappingTime):
        self.energy = energy
        self.startTime = startTime
        self.trappingTime = trappingTime

    def __repr__(self):
        return repr((self.energy, self.startTime, self.trappingTime))

for i in range(nDecays):
    # Get the direction and pitch angle of the decay electron
    theta = np.arccos(2 * np.random.rand() - 1)
    phi = 2 * np.pi * np.random.rand()
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    pa = np.arctan(np.sqrt(x**2 + y**2) / z)

    # Get energy if electron is trapped
    if abs(pa) > minPitchAngle:
        thisDecayE = np.interp(np.random.rand(), cdfDecayEnergy, energies)
        # Get the lifetime of the electron in the trap
        trappingMeanLifetime = bFit * 1e-6 + aFit * 1e-6 * thisDecayE
        
        trappingEvents.append(TrappingEvent(thisDecayE, np.random.rand() * timeWindow, 
                                            np.random.exponential(trappingMeanLifetime)))

In [ ]:
sortedByStartTime = sorted(trappingEvents, key=lambda x: x.startTime)

print(f"There are {len(sortedByStartTime)} total trapped electrons")

# Calculate the number of trapped electrons at each time
# Just look at the first 1000 events
changeTime = np.array([0.0])
nTrapped = np.array([0])
# Loop through the events and calculate when an electron is trapped or lost
for i in range(2000):
    # Get the event
    thisEvent = sortedByStartTime[i]
    # Add the creation time in ascending order in the array
    # Find the index where the time should be inserted    
    idx = np.searchsorted(changeTime, thisEvent.startTime)
    # Add the time where the number of electrons should change
    changeTime = np.insert(changeTime, idx, thisEvent.startTime)
    # Add the number of trapped electrons at this time
    nTrapped = np.insert(nTrapped, idx, nTrapped[idx - 1] + 1)
    # Increment the number of trapped electrons after this point
    nTrapped[idx + 1:] += 1

    # Now we want to figure out when the electron is lost
    # Find the index where the time should be inserted
    idxLost = np.searchsorted(changeTime, 
                              thisEvent.startTime + thisEvent.trappingTime)
    # Add the time where the number of electrons should change
    changeTime = np.insert(changeTime, idxLost, thisEvent.startTime + 
                           thisEvent.trappingTime)
    # Add the number of trapped electrons at this time
    nTrapped = np.insert(nTrapped, idxLost, nTrapped[idxLost - 1] - 1)
    # Decrement the number of trapped electrons after this point
    nTrapped[idxLost + 1:] -= 1

plt.figure()
plt.step(changeTime, nTrapped)
plt.xlabel('Time [s]')
plt.ylabel('Number of trapped electrons')
plt.xlim([0, sortedByStartTime[2000].startTime])
plt.show()